In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
b=True
while(b):
    print("1. Analyze Data")
    print("2. Edit Data")
    print("3. Sort Folder")
    print("4. Exit")
    choice=input("Enter Choice: ")
    if(choice=="1"):
        print(1)
        pass
    elif(choice=="2"):
        print(2)
        pass
    elif(choice=="3"):
        print(3)
        pass
    elif(choice=="4"):
        print("Exiting Program...")
        b=False
    else:
        print("Please Enter Valid Choice !")


In [ ]:
def Analyze_Data():
    b=True
    while(b):
        print("1. Analyze CSV File")
        print("2. Analyze Parquet File")
        print("3. Analyze Excel File")
        choice=input("Enter Choice: ")
        if(choice=="1"):
            print(1)
            pass
        elif(choice=="2"):
            print(2)
            pass
        elif(choice=="3"):
          print(3)
          pass
        elif(choice=="4"):
          print("Exiting Program...")
          b=False
        else:
          print("Please Enter Valid Choice !")
